# 🔹 Feature Engineering

في هذا الجزء قمنا بإنشاء خصائص جديدة (Engineered Features) من البيانات الأصلية بهدف تحسين دقة النموذج وتحليل العلاقة بينها وبين سعر البيع (SalePrice).

## 1. إنشاء خصائص جديدة (New Features)

- **HouseAge** = YrSold - YearBuilt (عمر المنزل منذ البناء حتى سنة البيع)  
- **RemodAge** = YrSold - YearRemodAdd (العمر منذ آخر تجديد)  
- **TotalSqFt** = 1stFlrSF + 2ndFlrSF + TotalBsmtSF (المساحة الكلية للمبنى)  
- **TotalBaths** = FullBath + HalfBath + BsmtFullBath + BsmtHalfBath (إجمالي عدد الحمامات)  
- **TotalPorchSF** = OpenPorchSF + EnclosedPorch + 3SsnPorch + ScreenPorch (إجمالي مساحة الشُرفات)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load training dataset
df = pd.read_csv("train.csv")

# 1. Create New Features
df["HouseAge"] = df["YrSold"] - df["YearBuilt"]
df["RemodAge"] = df["YrSold"] - df["YearRemodAdd"]
df["TotalSqFt"] = df["1stFlrSF"] + df["2ndFlrSF"] + df["TotalBsmtSF"]
df["TotalBaths"] = df["FullBath"] + df["HalfBath"] + df["BsmtFullBath"] + df["BsmtHalfBath"]
df["TotalPorchSF"] = df["OpenPorchSF"] + df["EnclosedPorch"] + df["3SsnPorch"] + df["ScreenPorch"]

df[["HouseAge","RemodAge","TotalSqFt","TotalBaths","TotalPorchSF"]].head()

## 2. تحليل الارتباط (Correlation Analysis)

قمنا بحساب معامل الارتباط (Correlation) بين الخصائص الجديدة وسعر البيع. نلاحظ أن:

- **TotalSqFt** له ارتباط قوي موجب مع SalePrice.  
- **OverallQual** من أقوى المؤشرات على السعر.  
- **HouseAge** له ارتباط ضعيف سلبي (المنازل الأقدم تميل لأن تكون أرخص).

In [ ]:
# Correlation with SalePrice
corr = df.corr()["SalePrice"].sort_values(ascending=False)
print("Top correlations with SalePrice:\n", corr.head(15))
print("\nLowest correlations with SalePrice:\n", corr.tail(15))

# Heatmap of top correlations
top_features = corr.head(11).index
plt.figure(figsize=(10,8))
sns.heatmap(df[top_features].corr(), annot=True, cmap="viridis", fmt=".2f")
plt.title("Correlation Heatmap of Top Features with SalePrice")
plt.show()

## 3. Visualizations

### Scatter Plots
- **TotalSqFt vs SalePrice**: يوضح أن المساحات الأكبر ترتبط بأسعار أعلى.  
- **HouseAge vs SalePrice**: يوضح أن المنازل الأقدم غالبًا أسعارها أقل.

In [ ]:
# Scatter: TotalSqFt vs SalePrice
plt.figure(figsize=(6,4))
sns.scatterplot(x="TotalSqFt", y="SalePrice", data=df, alpha=0.5)
plt.title("TotalSqFt vs SalePrice")
plt.show()

# Scatter: HouseAge vs SalePrice
plt.figure(figsize=(6,4))
sns.scatterplot(x="HouseAge", y="SalePrice", data=df, alpha=0.5, color="orange")
plt.title("HouseAge vs SalePrice")
plt.show()

### Boxplot (Neighborhood vs SalePrice)

- الحي (Neighborhood) من أهم الخصائص المؤثرة على السعر.  
- بعض الأحياء تتميز بمتوسط أسعار أعلى بكثير من غيرها (مثل NridgHt, StoneBr, NoRidge).

In [ ]:
plt.figure(figsize=(12,6))
order = df.groupby("Neighborhood")["SalePrice"].median().sort_values().index
sns.boxplot(x="Neighborhood", y="SalePrice", data=df, order=order)
plt.xticks(rotation=90)
plt.title("Neighborhood vs SalePrice")
plt.show()

## 📌 Insights

- المساحة الكلية (TotalSqFt) والجودة العامة (OverallQual) من أقوى العوامل المحددة لسعر البيع.  
- عمر التجديد (RemodAge) والعمر الكلي (HouseAge) يظهران علاقة عكسية ضعيفة.  
- الموقع (Neighborhood) يظل أحد أهم العوامل غير العددية المؤثرة بشكل كبير على السعر.